In [54]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.optimizers import Adam



#veriyi okuyoruz
veri = pd.read_csv("/content/car_price_prediction.csv")
veri = veri.drop(columns = "ID")

#eksik değer var mı ona bakıyoruz
#veri.isnull().sum()

#levy(harç) object olarak tanımlanmış özünde sayısal değer baktığımızda "-" olduğunu gördük bunu replace yapıcaz

veri["Levy"] = veri["Levy"].replace("-", np.nan)
veri["Levy"] = veri["Levy"].fillna(0)
veri["Levy"] = veri["Levy"].astype(int)
#veri.info()

veri["Turbo"] = veri["Engine volume"].apply(lambda x: "Turbo" if "Turbo" in str(x) else "Non Trurbo")
veri["Engine volume"] = veri["Engine volume"].apply(lambda x:str(x).replace("Turbo", "")).astype(float)
veri["Mileage"] = veri["Mileage"].apply(lambda x:str(x).replace("km", "")).astype(int)
veri["Cylinders"] = veri["Cylinders"].astype(int)

veri["Doors"] =np.where(veri["Doors"] == "04-May", 4, veri["Doors"])
veri["Doors"] =np.where(veri["Doors"] == "02-Mar", 2, veri["Doors"])
veri["Doors"] =np.where(veri["Doors"] == ">5", 5, veri["Doors"])
veri["Doors"] =veri["Doors"].astype(int)

#elimizdeki kategorik değişkenleri encoding uygulayarak sayısal değere çeviriyoruz
cat =[]
for i in veri.columns:
  if veri[i].dtype == "object":
    cat.append(i)

le = LabelEncoder()

for i in cat:
  veri[i]=le.fit_transform(veri[i])

y =veri["Price"]
x= veri.drop(columns="Price")

sc = StandardScaler()
x=sc.fit_transform(x)

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size=0.1,random_state=0)


def modelkur(units=64,activation="linear",learning_rate= 0.01,hidden_layers=2, dropout_rate=0.2):

  model=Sequential()
  model.add(Dense(units=units,input_dim=x_train.shape[1],activation=activation))

  for i in range(hidden_layers):
    model.add(Dense(units=units,activation=activation))
    model.add(Dropout(dropout_rate))

  model.add(Dense(1,activation=activation))
  model.compile(optimizer=Adam(learning_rate),loss="mae",metrics=["mae"])
  return model

parametreler={
    "units" : [32,64],
    "activation":["linear","relu"],
    "learning_rate":[0.01,0.001],
    "hidden_layers":[2,3],
    "dropout_rate":[0.2,0.3],
    "epochs":[50,100]
}

regres =  KerasRegressor(build_fn= modelkur,verbose=0)
grid= GridSearchCV(estimator=regres,param_grid=parametreler,cv=3)
gridsonuc=grid.fit(x_train,y_train)
bestparam = gridsonuc.best_params_
bestmodel = modelkur(units=bestparam["units"],activation=bestparam["activation"],learning_rate=bestparam["learning_rate"],
                     hidden_layers= bestparam["hidden_layers"],dropout_rate=bestparam["dropout_rate"])



cıktı = bestmodel.fit(x_train,y_train,epochs=bestparam["epochs "],verbose=0,validation_data=(x_val,y_val),batch_size=64)
tahmin = bestmodel.predict(x_test)

fig,ax = plt.subplots(1,2,figsize=(25,10))
ax[0].plot(cıktı.history["loss"],label="Training Loss")
ax[0].plot(cıktı.history["val_loss"],label="Validation Loss")
ax[0].set_title("loss Grafiği")
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Loss")
ax[0].legend()

ax[1].plot(range(len(y_test[:50])),y_test[:50],label="Gerçek")
ax[1].plot(range(len(y_test[:50])),y_test[:50],label="Tahmin")
ax[1].set_title("Araç Fiyat Tahmini")
ax[1].set_xlabel("Gözlem no")
ax[1].set_ylabel("Fiyat")
ax[1].legend()



ModuleNotFoundError: No module named 'tensorflow.keras.wrappers'